In [1]:
# add path so notebook can findd pyspark package

import os
USER = os.getenv('USER')

import sys
sys.path += ['', '/share/apps/pyspark/2.4.7/lib/python3.7/site-packages', 
             '/share/apps/anaconda3/2020.02/lib/python37.zip', 
             '/share/apps/anaconda3/2020.02/lib/python3.7', 
             '/share/apps/anaconda3/2020.02/lib/python3.7/lib-dynload', 
             '/home/{}/.local/lib/python3.7/site-packages'.format(USER), 
             '/share/apps/anaconda3/2020.02/lib/python3.7/site-packages']


from pyspark.sql import SparkSession
import json

import sys
from pprint import pprint
from datetime import timedelta, date, datetime

PATH_IN = "file:///scratch/work/twitter_decahose/daily/tweets/"

# Important: set JAVA_HOME so pyspark can start
os.environ['JAVA_HOME'] = '/share/apps/jdk/1.8.0_271'

# load CONFIG
CONFIG = json.load(open('config.json', 'r'))
FILTER_KEYWORD = '|'.join(CONFIG['KEYWORD'])
pprint(CONFIG)
pprint(FILTER_KEYWORD)

{'DASHBOARD_DIRECTORY_PREFIX': 'dashboard',
 'DASHBOARD_USER_NAME': 'csmap_user',
 'DASHBOARD_USER_PASSWORD': 'csmap',
 'DATABASE_NAME': 'tweet',
 'KEYWORD': ['new york', 'nyc', 'manhattan'],
 'TABLE_NAME': 'covid_tweet'}
'new york|nyc|manhattan'


In [2]:
# NOTE: a local mysql server has to be started
# pip install mysql-connector-python

import mysql.connector
cnx = mysql.connector.connect(user=CONFIG['DASHBOARD_USER_NAME'], 
                              password=CONFIG['DASHBOARD_USER_PASSWORD'],
                              host='localhost',
                              database=CONFIG['DATABASE_NAME'], 
                              charset='utf8',
                              use_unicode=True,
                             )

cursor = cnx.cursor()

add_tweet = ("""INSERT IGNORE INTO {}             
            (tweet_id, 
            tweet_timestamp,
            created_at,
            is_deleted,
            text,
            full_text,
            orig_user_id,
            source,
            truncated,
            in_reply_to_status_id,
            in_reply_to_user_id,
            in_reply_to_screen_name,
            coordinates,
            quote_count,
            reply_count,
            retweet_count,
            favorite_count,
            lang,
            user__id,
            user__name,
            user__screen_name,
            user__location,
            user__derived_location_country1,
            user__derived_location_country_code1,
            user__derived_location_region1,
            user__derived_location_sub_region1,
            user__derived_location_locality1,
            user__derived_location_full_name1,
            user__derived_location_geo1,
            user__url,
            user__description,
            user__followers_count,
            user__friends_count,
            user__listed_count,
            user__favorites_count,
            user__statuses_count,
            user__created_at,
            entities__hashtags,
            entities__urls,
            entities__expanded_urls,
            entities__user_mentions__id,
            entities__user_mentions__name,
            entities__user_mentions__screen_name,
            entities__symbols,
            entities__entities_polls,
            entities__media__media_urls,
            entities__media__urls,
            entities__media__display_urls,
            entities__media__expanded_urls,
            entities__media__types,
            extended_entities__media_urls,
            extended_entities__urls,
            extended_entities__display_urls,
            extended_entities__expanded_urls,
            extended_entities__types,
            quoted__created_at,
            quoted__id,
            quoted__text,
            quoted__full_text,
            quoted__in_reply_to_status_id,
            quoted__in_reply_to_user_id,
            quoted__in_reply_to_screen_name,
            quoted__quote_count,
            quoted__reply_count,
            quoted__retweet_count,
            quoted__favorite_count,
            quoted__lang,
            quoted__user__id,
            quoted__user__name,
            quoted__user__screen_name,
            quoted__user__location,
            quoted__user__url,
            quoted__user__description,
            quoted__user__followers_count,
            quoted__user__friends_count,
            quoted__user__listed_count,
            quoted__user__favorites_count,
            quoted__user__statuses_count,
            quoted__user__created_at,
            retweeted__created_at,
            retweeted__id,
            retweeted__text,
            retweeted__full_text,
            retweeted__in_reply_to_status_id,
            retweeted__in_reply_to_user_id,
            retweeted__in_reply_to_screen_name,
            retweeted__quote_count,
            retweeted__reply_count,
            retweeted__retweet_count,
            retweeted__favorite_count,
            retweeted__lang,
            retweeted__user__id,
            retweeted__user__name,
            retweeted__user__screen_name,
            retweeted__user__location,
            retweeted__user__url,
            retweeted__user__description,
            retweeted__user__followers_count,
            retweeted__user__friends_count,
            retweeted__user__listed_count,
            retweeted__user__favorites_count,
            retweeted__user__statuses_count,
            retweeted__user__created_at,
            when_created,
            label_1,
            label_2,
            label_3,
            label_4,
            label_5,
            reserved_1,
            reserved_2,
            reserved_3,
            reserved_4,
            reserved_5,
            yymmdd)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""".
             format(CONFIG['TABLE_NAME']))



In [3]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def transfer_one_day(date, spark_session):
    path_current = "{}{}/{}".format(PATH_IN, date, '*.parquet')
    print('path_current: ', path_current)

    df = spark_session.read.parquet(path_current)
    df.createOrReplaceTempView(CONFIG['TABLE_NAME'])
    df_s = spark_session.sql('''
        SELECT tweet_id, 
            tweet_timestamp,
            created_at,
            is_deleted,
            text,
            full_text,
            orig_user_id,
            source,
            truncated,
            in_reply_to_status_id,
            in_reply_to_user_id,
            in_reply_to_screen_name,
            coordinates,
            quote_count,
            reply_count,
            retweet_count,
            favorite_count,
            lang,
            user__id,
            user__name,
            user__screen_name,
            user__location,
            user__derived_location_country1,
            user__derived_location_country_code1,
            user__derived_location_region1,
            user__derived_location_sub_region1,
            user__derived_location_locality1,
            user__derived_location_full_name1,
            user__derived_location_geo1,
            user__url,
            user__description,
            user__followers_count,
            user__friends_count,
            user__listed_count,
            user__favorites_count,
            user__statuses_count,
            user__created_at,
            entities__hashtags,
            entities__urls,
            entities__expanded_urls,
            entities__user_mentions__id,
            entities__user_mentions__name,
            entities__user_mentions__screen_name,
            entities__symbols,
            entities__entities_polls,
            entities__media__media_urls,
            entities__media__urls,
            entities__media__display_urls,
            entities__media__expanded_urls,
            entities__media__types,
            extended_entities__media_urls,
            extended_entities__urls,
            extended_entities__display_urls,
            extended_entities__expanded_urls,
            extended_entities__types,
            quoted__created_at,
            quoted__id,
            quoted__text,
            quoted__full_text,
            quoted__in_reply_to_status_id,
            quoted__in_reply_to_user_id,
            quoted__in_reply_to_screen_name,
            quoted__quote_count,
            quoted__reply_count,
            quoted__retweet_count,
            quoted__favorite_count,
            quoted__lang,
            quoted__user__id,
            quoted__user__name,
            quoted__user__screen_name,
            quoted__user__location,
            quoted__user__url,
            quoted__user__description,
            quoted__user__followers_count,
            quoted__user__friends_count,
            quoted__user__listed_count,
            quoted__user__favorites_count,
            quoted__user__statuses_count,
            quoted__user__created_at,
            retweeted__created_at,
            retweeted__id,
            retweeted__text,
            retweeted__full_text,
            retweeted__in_reply_to_status_id,
            retweeted__in_reply_to_user_id,
            retweeted__in_reply_to_screen_name,
            retweeted__quote_count,
            retweeted__reply_count,
            retweeted__retweet_count,
            retweeted__favorite_count,
            retweeted__lang,
            retweeted__user__id,
            retweeted__user__name,
            retweeted__user__screen_name,
            retweeted__user__location,
            retweeted__user__url,
            retweeted__user__description,
            retweeted__user__followers_count,
            retweeted__user__friends_count,
            retweeted__user__listed_count,
            retweeted__user__favorites_count,
            retweeted__user__statuses_count,
            retweeted__user__created_at,
            when_created
        FROM {}
        WHERE (LOWER(text) RLIKE '({})' OR LOWER(full_text) RLIKE '({})')
    '''.format(
        CONFIG['TABLE_NAME'], FILTER_KEYWORD, FILTER_KEYWORD
    ))

    last_record = None

    for index, row in enumerate(df_s.rdd.toLocalIterator()):
        all_values = list(row.asDict().values())
        all_values = all_values + [None] * 10 + [date]
        last_record = tuple(all_values)
        
        cursor.execute(add_tweet, last_record)
            
        if index > 0 and index % 1000 == 0:
            pprint("inserted {} records...".format(index))
            cnx.commit()


def transfer_most_recent_3_days():
    end_date = date.today() - timedelta(days=2)
    start_date = end_date - timedelta(days=3)
    print(start_date)
    print(end_date)

    # init spark session
    spark_session = SparkSession.builder.master("local").appName("dashboard_daily_update").getOrCreate()

    for every_day in daterange(start_date, end_date):
        every_day = every_day.strftime("%Y-%m-%d")
        print('current date is {}'.format(every_day))
        try:
            transfer_one_day(date=every_day, spark_session=spark_session)
        except Exception as e:
            print('[Stop program], encountered an exception during INSERT...')
            print(e)
            cnx.commit()
            cursor.close()
            cnx.close()
            raise(e)

    # final commit
    cnx.commit()
    cursor.close()
    cnx.close()
    print('job finished!')
    
transfer_most_recent_3_days()

2021-03-24
2021-03-27
current date is 2021-03-24
path_current:  file:///scratch/work/twitter_decahose/daily/tweets/2021-03-24/*.parquet
'inserted 1000 records...'
'inserted 2000 records...'
'inserted 3000 records...'
'inserted 4000 records...'
'inserted 5000 records...'
'inserted 6000 records...'
'inserted 7000 records...'
'inserted 8000 records...'
'inserted 9000 records...'
'inserted 10000 records...'
'inserted 11000 records...'
'inserted 12000 records...'
'inserted 13000 records...'
'inserted 14000 records...'
'inserted 15000 records...'
'inserted 16000 records...'
'inserted 17000 records...'
'inserted 18000 records...'
'inserted 19000 records...'
'inserted 20000 records...'
'inserted 21000 records...'
'inserted 22000 records...'
'inserted 23000 records...'
'inserted 24000 records...'
'inserted 25000 records...'
'inserted 26000 records...'
'inserted 27000 records...'
'inserted 28000 records...'
'inserted 29000 records...'
'inserted 30000 records...'
'inserted 31000 records...'
'inse